<a href="https://colab.research.google.com/github/c-quilo/RL-OpenAIGym/blob/main/RL_QLearningNN_FrozenLakeNoSlip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reinforcement learning exercises using OpenAI Gym
Author: César Quilodrán-Casas
Environment: Cart Pole v1 *italicised text*



In [2]:
!pip install gym
!pip install ipython

In [16]:
#Load packages
import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time
from gym.envs.registration import register
from IPython.display import clear_output

Instructions for updating:
non-resource variables are not supported in the long term


In [17]:
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

In [18]:
envName = 'CartPole-v1'
envName = 'MountainCar-v0'
envName = 'MountainCarContinuous-v0'
envName = 'FrozenLakeNoSlip-v0'
env = gym.make(envName)
print('Observations:', env.observation_space)
print('Actions: ', env.action_space)
type(env.action_space)

Observations: Discrete(16)
Actions:  Discrete(4)


gym.spaces.discrete.Discrete

In [19]:
#Define Agent

class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print('Action_size: ', self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print('Action_range: ', self.action_low, self.action_high)
#Perform an action
    def get_action(self, state):
        if self.is_discrete:
            action = np.random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                        self.action_high,
                                        self.action_shape)
        return action

In [21]:
class QNNAgent(Agent):
    def __init__(self, env, discount_rate=0.95, learning_rate=0.1):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print('State size: ', self.state_size)

        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def build_model(self):
        self.state_in = tf.placeholder(tf.int32, shape=[1]) 
        self.action_in = tf.placeholder(tf.int32, shape=[1])
        self.target_in = tf.placeholder(tf.float32, shape=[1])

        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)

        self.q_state = tf.layers.dense(self.state, units=self.action_size)
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)

        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)

    def get_action(self, state):
      q_state = self.q_table[state]
      #Choose between an action from the q-table and a random action for the agent to explore
      action_greedy = np.argmax(q_state)
      action_random = super().get_action(state)
      return action_random if np.random.random() < self.eps else action_greedy

      return action
    
    def train(self, experience):
      state, action, next_state, reward, done = experience
      q_next = self.q_table[next_state]
      q_next = np.zeros([self.action_size]) if done else q_next
      q_target = reward + self.discount_rate * np.max(q_next)

      q_update = q_target - self.q_table[state, action]
      self.q_table[state, action] += self.learning_rate * q_update

      if done:
        self.eps = self.eps * 0.99
    
    def __del__(self):
      self.sess.close()
    
agent = QAgent(env)
    

Action_size:  4
State size:  16


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
#Training
#Maximises reward
total_reward = 0
for episode in range(100):
  state = env.reset()
  done = False
  while not done:
    action = agent.get_action(state)
    next_state, reward, done, info = env.step(action)
    agent.train((state, action, next_state, reward, done))
    state = next_state
    total_reward += reward
    print('s: ', state, 'a: ', action)
    print('Episode: {}, Total reward: {}, epsilon: {}'.format(episode, total_reward, agent.eps))
    env.render()
    print(agent.q_table)
    time.sleep(0.05)
    clear_output(wait=True)

s:  12 a:  0
Episode: 99, Total reward: 12.0, epsilon: 0.36603234127322926
  (Left)
SFFF
FHFH
FFFH
HFFG
[[2.91440919e-04 8.71214745e-05 8.19162848e-03 8.40021514e-05]
 [7.43115725e-05 1.22568721e-05 2.81473544e-02 2.06665906e-03]
 [2.86599702e-03 7.39261672e-02 2.85902884e-04 2.19926490e-03]
 [1.96653082e-03 1.26222413e-05 1.24593193e-04 6.76594177e-05]
 [5.88403518e-05 6.49655913e-05 4.32237813e-06 4.97434186e-04]
 [9.86276052e-05 8.06435554e-05 9.83435040e-05 4.06658820e-05]
 [5.57330681e-05 1.91746526e-01 2.58832967e-05 2.01658457e-03]
 [9.15415030e-05 2.49688128e-05 9.87010950e-05 7.84364143e-05]
 [6.55820021e-05 2.04561497e-06 5.52787292e-05 1.07083816e-04]
 [6.26433193e-05 3.21534114e-05 2.15931612e-05 3.65026662e-05]
 [5.09617192e-05 3.67809589e-01 6.49385295e-05 1.69288608e-02]
 [3.48700751e-05 8.98671919e-05 7.55113923e-05 2.35553166e-05]
 [7.12331059e-05 7.06591614e-05 8.59991018e-06 9.94094805e-05]
 [1.96832446e-05 4.23524141e-05 9.19312244e-05 4.68439883e-05]
 [3.93893733e-